In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import re
import threading
from sksurgerynditracker.nditracker import NDITracker
import os
import random

In [2]:
TOOL_TIP_OFFSET = np.array([-304.5728,-0.3053,-0.1412, 1])

In [3]:
# Initialize the NDI tracker
SETTINGS = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,
    "romfiles": ["/Users/yizheng/Desktop/Dartmouth/lab work/ARPAH/Polaris Vega XT/tool_defs/medtronic_chicken_foot_960_556.rom"]
}
tracker = NDITracker(SETTINGS)
tracker.start_tracking()

PING 169.254.7.250 (169.254.7.250): 56 data bytes
64 bytes from 169.254.7.250: icmp_seq=0 ttl=64 time=0.604 ms

--- 169.254.7.250 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.604/0.604/0.604/nan ms


In [4]:
# Initialize vtk
reader = vtk.vtkSTLReader() # read stl file
reader.SetFileName("stl-models/medtronic_chicken_foot.STL") # set the file name
reader.Update() # update the reader

mapper = vtk.vtkPolyDataMapper() # create a mapper, a mapper is used to map the data to the graphics primitives
mapper.SetInputData(reader.GetOutput()) # set the input to the reader

tool_actor = vtk.vtkActor() # create an actor, an actor is used to display the data
tool_actor.SetMapper(mapper) # set the mapper to the actor

renderer = vtk.vtkRenderer() # create a renderer, a renderer is used to display the data
render_window = vtk.vtkRenderWindow() # create a render window, a render window is used to display the data
render_window.AddRenderer(renderer) # add the renderer to the render window
render_window.SetSize(2000, 1600) # set the size of the render window

render_window_interactor = vtk.vtkRenderWindowInteractor() # create a render window interactor, a render window interactor is used to interact with the data
render_window_interactor.SetRenderWindow(render_window) # set the render window to the render window interactor

renderer.AddActor(tool_actor) # add the actor to the renderer
renderer.SetBackground(1, 1, 1) # set the background of the renderer

# 创建坐标轴
axes = vtk.vtkAxesActor()
axes.SetTotalLength(200, 200, 200)  # 设置坐标轴的长度
axes.SetAxisLabels(True)

# 设置坐标轴标签的位置
axes.GetXAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetYAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetZAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()

# 设置坐标轴标签
axes.GetXAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(1, 0, 0)  # X轴为红色
axes.GetYAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 1, 0)  # Y轴为绿色
axes.GetZAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 0, 1)  # Z轴为蓝色

# 将坐标轴添加到 renderer
renderer.AddActor(axes)

# 设定摄像机视角，固定在 (0, 0, -3000) 位置
renderer.GetActiveCamera().SetPosition(-1000, 1000, -5000)  # 摄像机位置
renderer.GetActiveCamera().SetFocalPoint(0, 0, -2000)  # 摄像机焦点
renderer.GetActiveCamera().SetViewUp(-1, 0, 0)  # 设置“上”方向
renderer.GetActiveCamera().SetClippingRange(1, 3000)  # 设置裁剪范围

In [5]:
cwd = os.getcwd()
models = []
for file in os.listdir(cwd + '/' + 'slicer_files/'):
    if file.endswith('.vtk') or file.endswith('.stl'):
        models.append(file)

print(models)

['RK_1.vtk', 'Box.vtk', 'IN_1.stl', 'IN.stl', 'RK.vtk', 'Box_1.vtk']


In [6]:
transformation_matrix = np.array([[-1.30866820e-02,  9.98415042e-01,  5.47370336e-02,  1.25183027e+02],
                        [ 9.88019474e-01,  2.13301560e-02, -1.52848106e-01, -1.24802059e+02],
                        [-1.53773398e-01,  5.20809806e-02, -9.86732645e-01, -1.26559772e+03],
                        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [7]:
# Loop through model files, load them, and add them to the renderer
for file in models:
    if file.endswith(".stl"):
        reader = vtk.vtkSTLReader()
        reader.SetFileName(cwd + '/' + 'slicer_files/' + file)
    elif file.endswith(".vtk"):
        reader = vtk.vtkPolyDataReader()
        reader.SetFileName(cwd + '/' + 'slicer_files/' + file)
    else:
        continue  # Skip unsupported formats

    # Update the reader to load the data
    reader.Update()

    # Create a mapper and actor to display the model
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(reader.GetOutput())
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(random.randint(0, 255)/255, random.randint(0, 255)/255, random.randint(0, 255)/255)

    transform = vtk.vtkTransform()
    transform.SetMatrix(transformation_matrix.flatten()) 
    actor.SetUserTransform(transform)


    if file.startswith("Box"):
        actor.GetProperty().SetOpacity(0.05)  # 设置透明度 (0.0 - 完全透明, 1.0 - 完全不透明)
    if file.startswith("Box_1"):
        actor.GetProperty().SetOpacity(0.00)  # 设置透明度 (0.0 - 完全透明, 1.0 - 完全不透明)
    # Add the actor to the renderer
    renderer.AddActor(actor)

In [8]:
def get_tracking_data(tracker):
    """Fetch latest tracking data"""
    port_handles, timestamps, framenumbers, transformation, quality = tracker.get_frame()
    transformation = np.array(transformation[0])
    return timestamps, transformation, quality

In [9]:
# Polaris Data Streaming Thread
data_ready = threading.Event()

def polaris_thread():
    global polaris_matrix
    while True:
        _, new_matrix, _ = get_tracking_data(tracker)
        polaris_matrix = new_matrix
        data_ready.set()
        time.sleep(1/400)

In [10]:
# VTK Update Thread
def update_pose(obj, event):
    global polaris_matrix

    if data_ready.is_set():
        transform = vtk.vtkTransform()
        transform.SetMatrix(polaris_matrix.flatten()) 
        tool_actor.SetUserTransform(transform)
        render_window.Render()
        data_ready.clear()

: 

In [ ]:
_, transformation, _ = get_tracking_data(tracker)

threading.Thread(target=polaris_thread, daemon=True).start()

render_window_interactor.AddObserver("TimerEvent", update_pose)
render_window_interactor.CreateRepeatingTimer(1)  # VTK 线程不断监听

render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.Start()

2025-02-21 17:29:29.840 python[27011:3489205] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-21 17:29:29.840 python[27011:3489205] +[IMKInputSession subclass]: chose IMKInputSession_Modern
